In [1]:
from acados_settings import acados_settings
from model.FW_lateral_model import FixedWingLateralModel
import utils as u
import numpy as np
from Paths.curve import Path
from Paths.waypoints import path_points

In [1]:
def call_mpcc(previous_x, previous_u, ocp_solver, current_state, params, N_horizon):
    # From an initial state x0 computes the optimal control input u_opt and the corresponding state trajectory 

    # Update MPC reference for all prediction steps
    for idx in range(N_horizon):
        ocp_solver.set(idx, 'x', previous_x[idx, :])
        ocp_solver.set(idx, 'u', previous_u[idx, :])
        ocp_solver.set(idx, 'p', params)

    # Set initial state
    ocp_solver.set(0, 'lbx', current_state)
    ocp_solver.set(0, 'ubx', current_state)

    # Solve MPC problem
    status = ocp_solver.solve()
    if status != 0:
        raise Exception(f'acados returned status {status}.')

    # Initialize matrices to store state and control trajectories
    X = np.zeros((N_horizon, previous_x.shape[1]))  # Assuming previous_x.shape[1] is the state dimension
    U = np.zeros((N_horizon, previous_u.shape[1]))  # Assuming previous_u.shape[1] is the control dimension

    # Retrieve the state and control trajectories from the solver
    for i in range(N_horizon):
        X[i, :] = ocp_solver.get(i, 'x')
        U[i, :] = ocp_solver.get(i, 'u')

    return X, U  # state and input for all the horizon (matrix)

In [ ]:
def warm_start(x0, ocp_solver, N_horizon, params):
    # Warm start the solver to generate the first trajectory point
    iter = 30
    optimal_x = np.zeros((N_horizon, 6))
    optimal_u = np.zeros((N_horizon, 4))
    for idx in range(iter):
        optimal_x, optimal_u = call_mpcc(optimal_x, optimal_u, ocp_solver, x0, params, N_horizon)

    return optimal_x, optimal_u

In [5]:
def main():
    # Create model instance
    model = FixedWingLateralModel()
    path = Path(path_points)
    # Initialize MPC solver
    N_horizon = 20
    Tf = 2.0
    # Initial state for MPC solver
    x0 = np.array([0.0, 0.0, 20.0, 0.0, 0.0, 0.0])  # initial state (x, y, V, yaw)
    ocp_solver, acados_integrator, mpc_dt = acados_settings(model, N_horizon, Tf, x0, use_RTI=False)
    # Lists to store state and input values for debugging
    state_history = []
    input_history = []
    # Generate reference points using Path class
    reference_history = path.spline_points
    simulation_time = 0
    max_simulation_time = 40.0

    params = np.zeros(2)  # Adjust size if needed

    current_state = x0.copy()  # Initialize current_state

    optimal_x, optimal_u = warm_start(x0, ocp_solver, N_horizon, params)
    print ("Warm start done")
    print("Optimal x: ", optimal_x, "Optimal u: ", optimal_u)


    previous_x, previous_u = optimal_x, optimal_u

    while simulation_time < max_simulation_time:
        
        x_opt, u_opt =  call_mpcc(previous_x, previous_u, ocp_solver, current_state, params, N_horizon)

        previous_x = x_opt
        previous_u = u_opt

        apply_control_input = u_opt[0,:]
        
        # simulate fixed wing
        current_state = acados_integrator.simulate(current_state, apply_control_input, mpc_dt)

        state_history.append(current_state)
        input_history.append(apply_control_input)
        simulation_time += mpc_dt

    u.plot_uav_trajectory_and_state(state_history, reference_history, input_history, params[:2])